In [3]:
import re
from dataclasses import dataclass

# الكلمات المفتاحية وفئاتها
categories = {
    "SUMMON": ("ProgramStructure", "Start"),
    "RECALL": ("ProgramStructure", "End"),
    "USE": ("ProgramStructure", "Import"),
    "BUILD": ("Variables", "Declaration"),
    "SAY": ("InputOutput", "Print"),
    "LISTEN": ("InputOutput", "Input"),
    "ULTIMATE": ("Functions", "Definition"),
    "CALL": ("Functions", "Call"),
    "BASE": ("Functions", "End"),
    "CHECK": ("ControlFlow", "If"),
    "MISS": ("ControlFlow", "Else"),
    "FARM": ("ControlFlow", "Loop"),
    "FF15": ("ControlFlow", "Break"),
}

# أنماط الرموز
keywords = "|".join(categories.keys())
patterns = {
    "COMMENT": r"//.*",
    "STRING": r'"(?:[^"\\]|\\.)*"',
    "NUMBER": r"\b\d+(?:\.\d+)?\b",
    "KEYWORD": rf"\b(?:{keywords})\b",
    "IDENTIFIER": r"\b[A-Za-z_]\w*\b",
    "SYMBOL": r"[{}\[\]();,\.]",
    "OPERATOR": r"(==|!=|<=|>=|[+\-*/=<>]|[+\-]{2})",
}




code = '''
USE DEMACIA
SUMMON GAME

BUILD CHAMPION name = "Garen"
BUILD POWER gold = 0
SAY "Welcome, " + name + "!"
CHECK (gold >= 1000) {
    SAY "You're rich!"
}
MISS {
    SAY "Keep FARMING!"
}

ULTIMATE SpinToWin() {
    SAY "SPIN TO WIN!"
    BASE
}

CALL SpinToWin
RECALL BASE
'''


In [4]:
token_re = re.compile("|".join(f"(?P<{k}>{v})" for k, v in patterns.items()))

@dataclass
class Token:
    type: str
    value: str
    category: str | None = None

def tokenize(code: str):
    for m in token_re.finditer(code):
        kind, val = m.lastgroup, m.group()
        if kind == "COMMENT" or val.isspace():
            continue
        cat = categories.get(val) if kind == "KEYWORD" else None
        yield Token(kind, val, cat)







def demo():

    for t in tokenize(code):
        if t.category:
            print(f"[{t.type:<10}] {t.value:<15} -> {t.category}")
        else:
            print(f"[{t.type:<10}] {t.value}")

if __name__ == "__main__":
    demo()

[KEYWORD   ] USE             -> ('ProgramStructure', 'Import')
[IDENTIFIER] DEMACIA
[KEYWORD   ] SUMMON          -> ('ProgramStructure', 'Start')
[IDENTIFIER] GAME
[KEYWORD   ] BUILD           -> ('Variables', 'Declaration')
[IDENTIFIER] CHAMPION
[IDENTIFIER] name
[OPERATOR  ] =
[STRING    ] "Garen"
[KEYWORD   ] BUILD           -> ('Variables', 'Declaration')
[IDENTIFIER] POWER
[IDENTIFIER] gold
[OPERATOR  ] =
[NUMBER    ] 0
[KEYWORD   ] SAY             -> ('InputOutput', 'Print')
[STRING    ] "Welcome, "
[OPERATOR  ] +
[IDENTIFIER] name
[OPERATOR  ] +
[STRING    ] "!"
[KEYWORD   ] CHECK           -> ('ControlFlow', 'If')
[SYMBOL    ] (
[IDENTIFIER] gold
[OPERATOR  ] >=
[NUMBER    ] 1000
[SYMBOL    ] )
[SYMBOL    ] {
[KEYWORD   ] SAY             -> ('InputOutput', 'Print')
[STRING    ] "You're rich!"
[SYMBOL    ] }
[KEYWORD   ] MISS            -> ('ControlFlow', 'Else')
[SYMBOL    ] {
[KEYWORD   ] SAY             -> ('InputOutput', 'Print')
[STRING    ] "Keep FARMING!"
[SYMBOL    ] }
[